In [1]:
import h5py
import rasterio
from rasterio.mask import mask
from rasterio.transform import from_origin
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd 
import pandas as pd
from dnb_annual import *
from variables import years, composites, ukr_region_map, pol_region_map
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [7]:
country_polygons = gpd.read_file('poland_boundaries.geojson')

In [8]:
country_polygons

# change names according to pol_region_map
country_polygons['name'] = country_polygons['name'].apply(lambda x: pol_region_map[x])

# rename name to shapeName
country_polygons.rename(columns={'name': 'shapeName'}, inplace=True)

In [9]:
country_polygons

,id,shapeName,density,path,geometry
0,5307,Dolnoslaskie,0,/world/Poland/Dolnośląskie,"MULTIPOLYGON (((16.16883 51.66089, 16.19236 51..."
1,5304,Kujawsko-Pomorskie,0,/world/Poland/Kujawsko-Pomorskie,"MULTIPOLYGON (((18.25566 53.69423, 18.27436 53..."
2,5315,Lodzkie,0,/world/Poland/Łódzkie,"MULTIPOLYGON (((19.34231 52.30918, 19.36254 52..."
3,5305,Lubelskie,0,/world/Poland/Lubelskie,"MULTIPOLYGON (((23.19990 52.29715, 23.19235 52..."
4,5308,Lubuskie,0,/world/Poland/Lubuskie,"MULTIPOLYGON (((16.02720 53.10739, 16.03948 53..."
5,5306,Malopolskie,0,/world/Poland/Małopolskie,"MULTIPOLYGON (((20.15906 50.43463, 20.17673 50..."
6,5309,Mazowieckie,0,/world/Poland/Mazowieckie,"MULTIPOLYGON (((21.56137 53.37360, 21.56842 53..."
7,5310,Opolskie,0,/world/Poland/Opolskie,"MULTIPOLYGON (((17.79365 51.13264, 17.79802 51..."
8,5313,Podkarpackie,0,/world/Poland/Podkarpackie,"MULTIPOLYGON (((22.00627 50.76723, 22.03055 50..."
9,5311,Podlaskie,0,/world/Poland/Podlaskie,"MULTIPOLYGON (((22.88482 54.42379, 22.88450 54..."


In [3]:
# this script is used only once to generate the regional images for each year
# Ukraine
ukr_polygons = gpd.read_file("geoBoundaries-UKR-ADM1.geojson")
# source: https://cartographyvectors.com/
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
for year in years:
    dnb = dnb_annual(year, composites, ukr_polygons, "ukr")
    dnb.load_all_data()
    dnb.filter_data()
    dnb.save_rasters()
    dnb.load_rasters()
    dnb.build_regional_images()
    dnb.add_padding()
    dnb.save_regional_images()

In [4]:
def clean_pol_polygons(pol_polygons):

    pol_polygons['name'] = pol_polygons['name'].apply(lambda x: pol_region_map[x])
    pol_polygons.rename(columns={'name': 'shapeName'}, inplace=True)

    return pol_polygons

    

In [5]:
# Poland
pol_polygons = gpd.read_file("poland_boundaries.geojson")
pol_polygons = clean_pol_polygons(pol_polygons)
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

for year in years:
    dnb = dnb_annual(year, composites, pol_polygons, "pol")
    dnb.load_all_data()
    dnb.filter_data()
    dnb.save_rasters()
    dnb.load_rasters()
    dnb.build_regional_images()
    dnb.add_padding()
    dnb.save_regional_images()



In [5]:
# this script is used to clean gdp data

# Inflation data
# inflation = pd.read_excel("data/isc_reg.xls", skiprows=2, header=1)
# inflation = inflation.drop(columns=inflation.columns[0])
# inflation = inflation.rename(columns={inflation.columns[-1]: "region"})
# inflation = inflation[~inflation["region"].isin(["Ukraine", "oblasts"])]
# inflation = inflation.dropna()
# inflation["region"] = inflation["region"].map(region_map)
# inflation.columns = inflation.columns.astype(str)
# inflation = inflation.melt(id_vars="region", var_name="year", value_name="inflation")
# inflation.to_csv("data/inflation.csv", index=False)

# GDP data
# gdp = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
# gdp = gdp.drop(columns=gdp.columns[0])
# gdp = gdp.iloc[:, np.r_[18:36, -1]]
# gdp = gdp.rename(columns={gdp.columns[-1]: "region"})
# gdp = gdp[~gdp["region"].isin(["Ukrane", "oblasts"])]
# gdp = gdp.dropna()
# gdp["region"] = gdp["region"].map(region_map)
# gdp["region"] = gdp["region"].fillna("Sevastopol")
# gdp.columns = gdp.columns.astype(str)
# gdp = gdp.rename(columns={gdp.columns[i]: gdp.columns[i][:4] for i in range(18)})
# gdp = gdp.melt(id_vars="region", var_name="year", value_name="real_gdp_change")

# # include only years from 2012 inclusive, exclude Sevastopol and the Autonomous Republic of Crimea
# gdp = gdp[gdp["year"].astype(int) >= 2012]
# gdp = gdp[~gdp["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]

# # set the value for the starting year to 100 (2012), NaN for the rest
# gdp.loc[gdp["year"] == "2012", "real_gdp"] = 100
# gdp = gdp.sort_values(by=["region", "year"])
# gdp["real_gdp_change"] = gdp["real_gdp_change"] / 100

# # reste the index
# gdp = gdp.reset_index(drop=True)

# # # calculate the real gdp
# for i in range(1, gdp.shape[0]):

#     # skip if the year is 2012
#     if gdp.loc[gdp.index[i], "year"] == "2012":
#         continue
#     else:
#         gdp.loc[gdp.index[i], "real_gdp"] = gdp.loc[gdp.index[i-1], "real_gdp"] * (gdp.loc[gdp.index[i], "real_gdp_change"])

# # delete the real_gdp_change column
# gdp = gdp.drop(columns="real_gdp_change")

# # get the nominal gdp
# gdp_nominal = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
# gdp_nominal = gdp_nominal.iloc[:, np.r_[9, -1]]
# gdp_nominal.columns = ["gdp_nominal", "region"]
# gdp_nominal = gdp_nominal[~gdp_nominal["region"].isin(["Ukrane", "oblasts"])]
# gdp_nominal = gdp_nominal.dropna()
# gdp_nominal["region"] = gdp_nominal["region"].map(region_map)
# gdp_nominal["region"] = gdp_nominal["region"].fillna("Sevastopol")

# # merge nominal gdp to real gdp by region
# gdp = gdp.merge(gdp_nominal, on="region")

# # multiple the real gdp by the nominal gdp
# gdp["real_gdp"] = gdp["real_gdp"] * gdp["gdp_nominal"]

# # drop the nominal gdp column
# gdp = gdp.drop(columns="gdp_nominal")

# # for the region column, change all spaces to _
# gdp["region"] = gdp["region"].str.replace(" ", "_")

# # save the data
# gdp.to_csv("data/clean_ukr_gdp.csv", index=False)


In [6]:
# Prepare gdp data, Ukraine
gdp_ukr = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp_ukr = gdp_ukr.drop(columns=gdp_ukr.columns[0])
gdp_ukr = gdp_ukr.iloc[:, np.r_[18:36, -1]]
gdp_ukr = gdp_ukr.rename(columns={gdp_ukr.columns[-1]: "region"})
gdp_ukr = gdp_ukr[~gdp_ukr["region"].isin(["Ukrane", "oblasts"])]
gdp_ukr = gdp_ukr.dropna()
gdp_ukr["region"] = gdp_ukr["region"].map(region_map)
gdp_ukr["region"] = gdp_ukr["region"].fillna("Sevastopol")
gdp_ukr.columns = gdp_ukr.columns.astype(str)
gdp_ukr = gdp_ukr.rename(columns={gdp_ukr.columns[i]: gdp_ukr.columns[i][:4] for i in range(18)})
gdp_ukr = gdp_ukr[~gdp_ukr["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]

# gdp_ukr_nominal = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
# gdp_ukr = gdp_ukr.drop(columns=gdp_ukr.columns[0])
# gdp_ukr = gdp_ukr.iloc[:, np.r_[18:36, -1]]

# select years from 2012 inclusive and the region column
gdp_ukr = gdp_ukr[[gdp_ukr.columns[i] for i in range(18) if int(gdp_ukr.columns[i]) >= 2012] + ["region"]]

# Poland
gdp_pol = pd.read_excel("data/pol_reg_gdp.xlsx", header=1, sheet_name=1)
gdp_pol = gdp_pol.drop(columns=gdp_pol.columns[0])
gdp_pol = gdp_pol.drop(0)
gdp_pol = gdp_pol.rename(columns={gdp_pol.columns[0]: "region"})
# gdp_pol = gdp_pol.drop(columns="2022")

def clean_gdp_data(data, country):

    # set the 2012 first column to 100
    data["2012"] = 100

    # divide columns from 2013 to 2021 by 100
    for year in range(2013, 2022):
        data[str(year)] = data[str(year)] / 100

    # calculate the real gdp
    for i in range(2013, 2022):
        data[str(i)] = data[str(i)] * data[str(i-1)]

    # add a column 2022 and 2023 with NaN
    if country == "ukr":
        data["2022"] = np.nan
        data["2023"] = np.nan

    # format to long
    data = data.melt(id_vars="region", var_name="year", value_name="real_gdp")
    data["region"] = data["region"].str.replace(" ", "_")
    
    return data

gdp_pol = clean_gdp_data(gdp_pol, country = "pol")
gdp_ukr = clean_gdp_data(gdp_ukr, country = "ukr")

# read nominal gdp data for Ukraine
gdp_ukr_nominal = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp_ukr_nominal = gdp_ukr_nominal.drop(columns=gdp_ukr_nominal.columns[0])
gdp_ukr_nominal = gdp_ukr_nominal.iloc[:, np.r_[8, -1]]

gdp_ukr_nominal = gdp_ukr_nominal.rename(columns={gdp_ukr_nominal.columns[-1]: "region"})
gdp_ukr_nominal = gdp_ukr_nominal[~gdp_ukr_nominal["region"].isin(["Ukrane", "oblasts"])]
gdp_ukr_nominal = gdp_ukr_nominal.dropna()
gdp_ukr_nominal["region"] = gdp_ukr_nominal["region"].map(region_map)
gdp_ukr_nominal["region"] = gdp_ukr_nominal["region"].fillna("Sevastopol")
gdp_ukr_nominal.columns = gdp_ukr_nominal.columns.astype(str)
gdp_ukr_nominal = gdp_ukr_nominal[~gdp_ukr_nominal["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]
gdp_ukr_nominal["region"] = gdp_ukr_nominal["region"].str.replace(" ", "_")

# merge nominal gdp to real gdp by region
gdp_ukr = gdp_ukr.merge(gdp_ukr_nominal, on="region")

# multiple the real gdp by the nominal gdp
gdp_ukr["real_gdp"] = gdp_ukr["real_gdp"] * gdp_ukr["2012"]

# drop the nominal gdp column
gdp_ukr = gdp_ukr.drop(columns="2012")

# save the data
gdp_pol.to_csv("data/clean_pol_gdp.csv", index=False)
gdp_ukr.to_csv("data/clean_ukr_gdp.csv", index=False)


In [7]:
gdp_ukr

,region,year,real_gdp
0,Vinnytsia_Oblast,2012,3302400.0
1,Volyn_Oblast,2012,2000500.0
2,Dnipropetrovsk_Oblast,2012,14797000.0
3,Donetsk_Oblast,2012,17077500.0
4,Zhytomyr_Oblast,2012,2484900.0
...,...,...,...
295,Khmelnytskyi_Oblast,2023,NaN
296,Cherkasy_Oblast,2023,NaN
297,Chernivtsi_Oblast,2023,NaN
298,Chernihiv_Oblast,2023,NaN


In [8]:

gdp_ukr_nominal = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp_ukr_nominal = gdp_ukr_nominal.drop(columns=gdp_ukr_nominal.columns[0])
gdp_ukr_nominal = gdp_ukr_nominal.iloc[:, np.r_[8, -1]]

gdp_ukr_nominal = gdp_ukr_nominal.rename(columns={gdp_ukr_nominal.columns[-1]: "region"})
gdp_ukr_nominal = gdp_ukr_nominal[~gdp_ukr_nominal["region"].isin(["Ukrane", "oblasts"])]
gdp_ukr_nominal = gdp_ukr_nominal.dropna()
gdp_ukr_nominal["region"] = gdp_ukr_nominal["region"].map(region_map)
gdp_ukr_nominal["region"] = gdp_ukr_nominal["region"].fillna("Sevastopol")
gdp_ukr_nominal.columns = gdp_ukr_nominal.columns.astype(str)
gdp_ukr_nominal = gdp_ukr_nominal[~gdp_ukr_nominal["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]
gdp_ukr_nominal["region"] = gdp_ukr_nominal["region"].str.replace(" ", "_")
gdp_ukr_nominal

,2012,region
3,33024.0,Vinnytsia_Oblast
4,20005.0,Volyn_Oblast
5,147970.0,Dnipropetrovsk_Oblast
6,170775.0,Donetsk_Oblast
7,24849.0,Zhytomyr_Oblast
8,21404.0,Zakarpattia_Oblast
9,54828.0,Zaporizhia_Oblast
10,32286.0,Ivano-Frankivsk_Oblast
11,69663.0,Kyiv_Oblast
12,22056.0,Kirovohrad_Oblast


### Neural Network

In [3]:
# # Loading the MNIST dataset
# from keras.datasets import mnist
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [6]:
# load clean gdp data, keep only year, region and real_gdp columns
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")

# delete observations with year > 2021, reset index
ukraine = ukraine[ukraine["year"].astype(int) <= 2021]
ukraine.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(ukraine), 765, 1076, 1))
y = np.zeros(len(ukraine))

# load the images
for i in range(len(ukraine)):

    # get year, region, and gdp
    year = ukraine["year"][i]
    region = ukraine["region"][i]
    gdp_value = ukraine["real_gdp"][i]

    # get the file name
    file_name = f"{year}_{region}_hq.h5"

    # load the image
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        # nearnad_snow_free = annual_region["NearNadir_Composite_Snow_Free"][:]
        # offnad_snow_free = annual_region["OffNadir_Composite_Snow_Free"][:]
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

        # add the two images together
        # combined = snow_covered + snow_free

    # add the gdp value to y
    y[i] = gdp_value

    # append images to X
    X[i, :, :, 0] = allangle_snow_free
    # X[i, :, :, 0] = offnad_snow_free
    # X[i, :, :, 0] = nearnad_snow_free

# normalise the images
# maximum = X.max()
# X = X / maximum

# get indices for observations with  year = 2021: this is the test set
test_indices = np.where(ukraine["year"].astype(int) == 2021)[0]
train_indices = np.where(ukraine["year"].astype(int) != 2021)[0]

# get the train and test sets
X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

In [3]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 1)))
# Start with Convolutional layers
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))  # Additional Conv layer
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))  # Additional Conv layer
# model.add(MaxPooling2D((2, 2)))
# Flatten the results to feed into a dense layer
model.add(Flatten())
# Add dense layers (hidden layers)
model.add(Dense(64, activation='relu'))  # Upscaled dense layer
model.add(Dense(32, activation='relu'))   # Additional dense layer
# Output layer
model.add(Dense(1))
# Compile the model
model.compile(optimizer=Adam(), loss='mean_absolute_error', metrics=['mae'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 300, 440, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 298, 438, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 219, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 217, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 108, 32)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 252288)            0

In [12]:
def basic_block(x, filters, stride=1):
    y = Conv2D(filters, (3, 3), strides=stride, padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv2D(filters, (3, 3), padding='same')(y)
    y = BatchNormalization()(y)

    if stride != 1 or x.shape[-1] != filters:
        x = Conv2D(filters, (1, 1), strides=stride, padding='same')(x)
    y = Add()([x, y])
    y = Activation('relu')(y)
    return y

def ResNet18(input_shape=(765, 1076, 1)):
    input = Input(shape=input_shape)

    x = Resizing(300, 440, input_shape=(765, 1076, 1))

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = basic_block(x, 32, stride=1)
    x = basic_block(x, 32, stride=1)
    x = basic_block(x, 64, stride=2)
    x = basic_block(x, 64, stride=1)
    x = basic_block(x, 128, stride=2)
    x = basic_block(x, 128, stride=1)
    # x = basic_block(x, 512, stride=2)
    # x = basic_block(x, 512, stride=1)

    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='linear')(x)

    model = Model(inputs=input, outputs=x)
    return model

# Example usage
model = ResNet18()
model.summary()

model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 765, 1076, 1)]       0         []                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 383, 538, 64)         3200      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_13 (Ba  (None, 383, 538, 64)         256       ['conv2d_24[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_13 (Activation)  (None, 383, 538, 64)         0         ['batch_normalization_13

In [7]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 1)))

# Start with Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

# Flatten the results to feed into a dense layer
model.add(Flatten())

# Add dense layers (hidden layers)
model.add(Dense(256, activation='relu'))  # Increased the number of neurons
model.add(Dense(128, activation='relu'))  # Increased the number of neurons
model.add(Dense(64, activation='relu'))   # Kept as is for detailed feature extraction

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [10]:
# check the size of X_train and y_train
print(X_train.shape)
print(X_train[1, :, :, 0].max())

(200, 765, 1076, 1)
0.01435


In [8]:
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)  # Assuming you have a validation split of 20%


Epoch 1/50


3/3 [==============================] - 27s 7s/step - loss: 2986162432.0000 - mae: 44922.8594 - val_loss: 16875475968.0000 - val_mae: 73818.3594
Epoch 2/50
3/3 [==============================] - 22s 7s/step - loss: 1649803392.0000 - mae: 31756.7383 - val_loss: 12461476864.0000 - val_mae: 61813.4570
Epoch 3/50
3/3 [==============================] - 25s 9s/step - loss: 852806400.0000 - mae: 16027.0098 - val_loss: 14683069440.0000 - val_mae: 62618.0234
Epoch 4/50
3/3 [==============================] - 21s 7s/step - loss: 585213440.0000 - mae: 17655.7324 - val_loss: 15069169664.0000 - val_mae: 63303.2773
Epoch 5/50
3/3 [==============================] - 26s 8s/step - loss: 338988576.0000 - mae: 12590.5137 - val_loss: 14138784768.0000 - val_mae: 64194.4492
Epoch 6/50
3/3 [==============================] - 19s 6s/step - loss: 403277824.0000 - mae: 14660.1377 - val_loss: 14464156672.0000 - val_mae: 63644.5938
Epoch 7/50
3/3 [==============================] - 24s 8s/step - loss: 20

KeyboardInterrupt: 

In [7]:
# Step 6: Evaluate your model on the testing data
test_loss, test_mae = model.evaluate(X_test, y_test)
print('Test MAE:', test_mae) # mean absolute error
print('Test Loss:', test_loss)

1/1 [==============================] - 0s 292ms/step - loss: 13331.5254 - mae: 13331.5254
Test MAE: 13331.525390625
Test Loss: 13331.525390625


In [8]:
y_hat = model.predict(X_test).flatten()

# print the mean absolute percentage error
print(np.mean(100*np.abs((y_test - y_hat) / y_test)))

1/1 [==============================] - 1s 541ms/step
15.93812160316615


In [10]:
# get the predictions from X_test
y_hat = model.predict(X_test).flatten()

# convert the predictions back to the original scale
# y_hat = y_hat * y_std + y_mean
# y_test = y_test * y_std + y_mean

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))
print(mae)

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))
print(percentage_error)


2/2 [==============================] - 1s 288ms/step
0.038987137
27.844175


## Autoencoder

In [1]:
import pandas as pd
import numpy as np
import h5py

gdp = pd.read_csv("data/clean_ukr_gdp.csv")


# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(gdp), 765, 1076))

# load the snow covered and snow free images, add them together and append to the list
for i in range(len(gdp)):

    # get year, region, and gdp
    year = gdp["year"][i]
    region = gdp["region"][i]
    gdp_value = gdp["real_gdp"][i]

    # get the file name
    file_name = f"{year}_{region}_hq.h5"

    # load the image
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        # nearnad_snow_cov = annual_region["NearNadir_Composite_Snow_Covered"][:]
        # nearnad_snow_free = annual_region["NearNadir_Composite_Snow_Free"][:]
        # offnad_snow_cov = annual_region["OffNadir_Composite_Snow_Covered"][:]
        offnad_snow_free = annual_region["OffNadir_Composite_Snow_Free"][:]
        # allangle_snow_cov = annual_region["AllAngle_Composite_Snow_Covered"][:]
        # allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

        # add the two images together
        # combined = snow_covered + snow_free

    # append both images as two channels to to X
    # X[i, :, :]= allangle_snow_cov
    # X[i, :, :] = allangle_snow_free
    # X[i, :, :] = offnad_snow_cov
    X[i, :, :] = offnad_snow_free
    # X[i, :, :] = nearnad_snow_cov
    # X[i, :, :] = nearnad_snow_free

X = X.astype(np.float32)

# take the log
X = np.log(X + 1)


In [2]:
# resize images 
import tensorflow as tf
print(X.shape)

h, w = 382, 538
if X.ndim == 3:  # If images have shape (300, 765, 1076)
    X = np.expand_dims(X, axis=-1)
images_resized = tf.image.resize(X, (h, w)).numpy()

print(images_resized.shape)

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



(300, 765, 1076)
(300, 382, 538, 1)


In [3]:
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Dimensions
input_shape = (h, w, 1)

# Length of the compressed vector
k = 10

# Resize the data
images_resized = tf.image.resize(X, (h, w)).numpy()

# normalize the data
# images_resized = images_resized / images_resized.max()

# Define the autoencoder
input_img = layers.Input(shape=input_shape)

# Encoder
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

# Flatten and add the bottleneck layer
x = layers.Flatten()(x)
encoded = layers.Dense(k, activation='relu')(x)

# Decoder
x = layers.Dense((h // 4) * (w// 4) * 32, activation='relu')(encoded)
x = layers.Reshape((h // 4, w // 4, 32))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)

decoded = layers.Conv2D(1, (3, 3), activation='linear', padding='same')(x)
# Create the autoencoder model
autoencoder = models.Model(input_img, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Print the model summary
autoencoder.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 382, 538, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 382, 538, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 191, 269, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 191, 269, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 96, 135, 32)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 414720)            0  

In [4]:
# Assuming you have training data loaded in `train_images`
autoencoder.fit(images_resized, images_resized, epochs=20, batch_size=32)

Epoch 1/20

10/10 [==============================] - 29s 2s/step - loss: 0.1712
Epoch 2/20
10/10 [==============================] - 19s 2s/step - loss: 0.1492
Epoch 3/20
10/10 [==============================] - 26s 3s/step - loss: 0.1378
Epoch 4/20
10/10 [==============================] - 29s 3s/step - loss: 0.1248
Epoch 5/20
10/10 [==============================] - 24s 2s/step - loss: 0.1127
Epoch 6/20
10/10 [==============================] - 26s 2s/step - loss: 0.1010
Epoch 7/20
10/10 [==============================] - 21s 2s/step - loss: 0.0895
Epoch 8/20
10/10 [==============================] - 19s 2s/step - loss: 0.0803
Epoch 9/20
10/10 [==============================] - 20s 2s/step - loss: 0.0730
Epoch 10/20
10/10 [==============================] - 21s 2s/step - loss: 0.0673
Epoch 11/20
10/10 [==============================] - 21s 2s/step - loss: 0.0638
Epoch 12/20
10/10 [==============================] - 22s 2s/step - loss: 0.0602
Epoch 13/20
10/10 [=============================

In [5]:
autoencoder.save('simple_autoencoder.h5')
encoder = models.Model(input_img, encoded)
encoder.save('encoder.h5')


c:\Users\jakub\anaconda\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
encoded_imgs = encoder.predict(images_resized)

10/10 [==============================] - 3s 165ms/step


In [45]:

# turn the encoded images to a pandas dataframe
encoded_imgs = pd.DataFrame(encoded_imgs)
gdp = pd.read_csv("data/clean_ukr_gdp.csv")


# change column names to encoded_1, encoded_2, ...
encoded_imgs.columns = [f"encoded_{i}" for i in range(encoded_imgs.shape[1])]

# # add encoded images to the gdp data as new columns
gdp = pd.concat([gdp, encoded_imgs], axis=1)

data = pd.get_dummies(gdp, columns=["region"])

# training data contains years until 2021
train_data = data[data["year"] <= 2021]
test_data = data[data["year"] >= 2022]

X = train_data.drop(columns=["year", "real_gdp"])
y = train_data["real_gdp"]

# take randomly 80% of the data for training
train_size = int(0.8 * len(train_data))
test_size = len(train_data) - train_size

train_indices = np.random.choice(len(train_data), train_size, replace=False)
test_indices = np.setdiff1d(np.arange(len(train_data)), train_indices)

X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]


In [17]:
# fit a XGBoost model
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=0)

xgb_model.fit(X_train, y_train)

# get the predictions
y_hat = xgb_model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

ValueError: Please reshape the input data into 2-dimensional matrix.

In [47]:
# predict on full data
best_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=0)

best_model.fit(X, y)

results = test_data
test_data = test_data.drop(columns=["year", "real_gdp"])
y_hat = best_model.predict(test_data)

results["gdp_prediction"] = y_hat

# results = results[["region", "gdp_prediction"]]


C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/4118865579.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results["gdp_prediction"] = y_hat


In [49]:
results 

,year,real_gdp,encoded_0,encoded_1,encoded_2,encoded_3,encoded_4,encoded_5,encoded_6,encoded_7,...,region_Poltava_Oblast,region_Rivne_Oblast,region_Sumy_Oblast,region_Ternopil_Oblast,region_Vinnytsia_Oblast,region_Volyn_Oblast,region_Zakarpattia_Oblast,region_Zaporizhia_Oblast,region_Zhytomyr_Oblast,gdp_prediction
250,2022,NaN,0.000000,32.356968,0.000000,43.147476,6.614171,57.424915,0.000000,0.000000,...,False,False,False,False,True,False,False,False,False,2.901053e+06
251,2022,NaN,0.000000,65.775826,0.000000,42.208607,28.832651,20.495667,0.000000,19.882025,...,False,False,False,False,False,True,False,False,False,2.009610e+06
252,2022,NaN,0.000000,0.000000,28.939548,0.000000,0.000000,33.387489,51.399616,0.000000,...,False,False,False,False,False,False,False,False,False,1.370453e+07
253,2022,NaN,85.156311,2.947619,62.387512,32.447998,23.472427,31.117155,4.903654,44.713188,...,False,False,False,False,False,False,False,False,False,3.709054e+06
254,2022,NaN,0.000000,6.884701,0.000000,38.491554,0.000000,7.141434,0.000000,0.568389,...,False,False,False,False,False,False,False,False,True,2.948006e+06
255,2022,NaN,0.000000,132.615051,0.000000,5.576831,13.221070,0.000000,0.000000,13.538671,...,False,False,False,False,False,False,True,False,False,2.111304e+06
256,2022,NaN,0.000000,16.066391,0.000000,0.000000,69.055626,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,True,False,2.851961e+06
257,2022,NaN,0.000000,0.000000,24.315319,12.990799,23.690477,40.577160,0.000000,30.253977,...,False,False,False,False,False,False,False,False,False,3.480044e+06
258,2022,NaN,0.000000,24.124853,11.860563,3.499140,0.000000,83.198761,0.000000,42.292961,...,False,False,False,False,False,False,False,False,False,2.524573e+06
259,2022,NaN,0.000000,0.000000,49.317535,23.669285,0.000000,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,2.799378e+06


In [18]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=1000, random_state = 0)
rf_model.fit(X_train, y_train)

# get the predictions
y_hat = rf_model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

ValueError: Found array with dim 4. Estimator expected <= 2.

In [13]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0)

model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)


620590.9893821041
13.228268150766247


C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/1003370021.py:5: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train, y_train)
c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 54890190761860.125, tolerance: 713662316497.8124
  model = cd_fast.enet_coordinate_descent(


## Single network per region

In [6]:
# define the model
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 1)))
# Start with Convolutional layers
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))  # Additional Conv layer
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))  # Additional Conv layer
# model.add(MaxPooling2D((2, 2)))
# Flatten the results to feed into a dense layer
model.add(Flatten())
# Add dense layers (hidden layers)
model.add(Dense(64, activation='relu'))  # Upscaled dense layer
model.add(Dense(32, activation='relu'))   # Additional dense layer
# Output layer
model.add(Dense(1))
# Compile the model
model.compile(optimizer=Adam(), loss='mean_absolute_error', metrics=['mae'])

In [7]:
# load clean gdp data
gdp = pd.read_csv("data/tabular_data_ukraine.csv")

# initialise a dictionary to store the results
results = {"region": [], "predicted_gdp": [], "actual_gdp": [], "mae": [], "percentage_error": []}

# delete observations with year > 2021
gdp = gdp[gdp["year"].astype(int) <= 2021]

# get unique regions and years
regions = gdp["region"].unique()
# regions = ['Cherkasy_Oblast']
years = gdp["year"].unique()

# load the snow covered and snow free images, add them together and append to the list
for region in regions:

    # initialise data
    X = np.zeros((len(years), 765, 1076, 1))
    X_test = np.zeros((1, 765, 1076, 1))
    y = np.zeros(len(years))

    for i in range(len(years)):

        year = years[i]
        gdp_value = gdp[(gdp["region"] == region) & (gdp["year"] == year)]["real_gdp"].values[0]

        # get the file name
        file_name = f"{year}_{region}_hq.h5"

        # load the image
        file_path = f"data/annual_region_images/{file_name}"
    
        with h5py.File(file_path, 'r') as annual_region:
            # nearnad_snow_cov = annual_region["NearNadir_Composite_Snow_Covered"][:]
            # nearnad_snow_free = annual_region["NearNadir_Composite_Snow_Free"][:]
            # offnad_snow_cov = annual_region["OffNadir_Composite_Snow_Covered"][:]
            # offnad_snow_free = annual_region["OffNadir_Composite_Snow_Free"][:]
            # allangle_snow_cov = annual_region["AllAngle_Composite_Snow_Covered"][:]
            allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

            # add the two images together
            # combined = snow_covered + snow_free

        # add the gdp value to y

        if year != 2021:
            y[i] = gdp_value

            # append both images as two channels to to X
            # X[i, :, :, 0] = allangle_snow_cov
            X[i, :, :, 0] = allangle_snow_free
            # X[i, :, :, 2] = offnad_snow_cov
            # X[i, :, :, 0] = offnad_snow_free 
            # X[i, :, :, 4] = nearnad_snow_cov
            # X[i, :, :, 1] = nearnad_snow_free 
        else:
            X_test[0, :, :, 0] = allangle_snow_free
            # X_test[0, :, :, 1] = nearnad_snow_free

            y_test = gdp_value

        # # Normalise the images
        # X[:, :, :, 0] = X[:, :, :, 0] / X[:, :, :, 0].max()
        # X[:, :, :, 1] = X[:, :, :, 1] / X[:, :, :, 1].max()

        # X_test[:, :, :, 0] = X_test[:, :, :, 0] / X[:, :, :, 0].max()
        # X_test[:, :, :, 1] = X_test[:, :, :, 1] / X[:, :, :, 1].max()

        # # normalise GDP values
        # y = y / y.max()
        # y_test = y_test / y.max()



    # fit the model on the data
    model.fit(X, y, epochs=50, batch_size=16, validation_split=0.2)  # Assuming you have a validation split of 20%

    # predict the gdp for 2021
    y_hat = model.predict(X_test).flatten()

    print(f"Predicted GDP for {region} in 2021: {y_hat}")
    print(f"Actual GDP for {region} in 2021: {y_test}")

    # add the results to the dictionary
    mae = np.mean(np.abs(y_test - y_hat))
    percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

    results["region"].append(region)
    results["predicted_gdp"].append(y_hat)
    results["actual_gdp"].append(y_test)
    results["mae"].append(mae)
    results["percentage_error"].append(percentage_error)


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 37222.0117 - mae: 37222.0117 - val_loss: 20516.8477 - val_mae: 20516.8477
Epoch 2/50
1/1 [==============================] - 0s 320ms/step - loss: 37163.3047 - mae: 37163.3047 - val_loss: 20474.7051 - val_mae: 20474.7051
Epoch 3/50
1/1 [==============================] - 0s 335ms/step - loss: 37072.5625 - mae: 37072.5625 - val_loss: 20423.6094 - val_mae: 20423.6094
Epoch 4/50
1/1 [==============================] - 0s 314ms/step - loss: 36961.8242 - mae: 36961.8242 - val_loss: 20362.8555 - val_mae: 20362.8555
Epoch 5/50
1/1 [==============================] - 0s 318ms/step - loss: 36828.3750 - mae: 36828.3750 - val_loss: 20289.2520 - val_mae: 20289.2520
Epoch 6/50
1/1 [==============================] - 0s 386ms/step - loss: 36664.7852 - mae: 36664.7852 - val_loss: 20201.1504 - val_mae: 20201.1504
Epoch 7/50
1/1 [==============================] - 0s 357ms/step - loss: 36466.7031 - mae: 36466.7031 - val_loss: 20095.2207 - v

C:\Users\jakub\AppData\Local\Temp/ipykernel_18280/568650546.py:77: RuntimeWarning: overflow encountered in cast
  y = y.astype(np.float16)
C:\Users\jakub\AppData\Local\Temp/ipykernel_18280/568650546.py:79: RuntimeWarning: overflow encountered in cast
  y_test = y_test.astype(np.float16)


1/1 [==============================] - 0s 439ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 2/50
1/1 [==============================] - 0s 287ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 3/50
1/1 [==============================] - 0s 285ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 4/50
1/1 [==============================] - 0s 288ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 5/50
1/1 [==============================] - 0s 296ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 6/50
1/1 [==============================] - 0s 282ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 7/50
1/1 [==============================] - 0s 288ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 8/50
1/1 [==============================] - 0s 291ms/step - loss: inf - mae: inf - val_loss: inf - val_mae: inf
Epoch 9/50
1/1 [==============================] - 0s 322ms/step - l

C:\Users\jakub\AppData\Local\Temp/ipykernel_18280/568650546.py:92: RuntimeWarning: invalid value encountered in divide
  percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))


Epoch 1/50
1/1 [==============================] - 0s 376ms/step - loss: inf - mae: inf - val_loss: 15289260.0000 - val_mae: 15289260.0000
Epoch 2/50
1/1 [==============================] - 0s 294ms/step - loss: inf - mae: inf - val_loss: 16316784.0000 - val_mae: 16316784.0000
Epoch 3/50
1/1 [==============================] - 0s 287ms/step - loss: inf - mae: inf - val_loss: 17361300.0000 - val_mae: 17361300.0000
Epoch 4/50
1/1 [==============================] - 0s 294ms/step - loss: inf - mae: inf - val_loss: 18424332.0000 - val_mae: 18424332.0000
Epoch 5/50
1/1 [==============================] - 0s 288ms/step - loss: inf - mae: inf - val_loss: 19507770.0000 - val_mae: 19507770.0000
Epoch 6/50
1/1 [==============================] - 0s 284ms/step - loss: inf - mae: inf - val_loss: 20613842.0000 - val_mae: 20613842.0000
Epoch 7/50
1/1 [==============================] - 0s 297ms/step - loss: inf - mae: inf - val_loss: 21745172.0000 - val_mae: 21745172.0000
Epoch 8/50
1/1 [==================

In [8]:
results

{'region': ['Vinnytsia_Oblast',
  'Volyn_Oblast',
  'Dnipropetrovsk_Oblast',
  'Donetsk_Oblast',
  'Zhytomyr_Oblast',
  'Zakarpattia_Oblast',
  'Zaporizhia_Oblast',
  'Ivano-Frankivsk_Oblast',
  'Kyiv_Oblast',
  'Kirovohrad_Oblast',
  'Luhansk_Oblast',
  'Lviv_Oblast',
  'Mykolaiv_Oblast',
  'Odessa_Oblast',
  'Poltava_Oblast',
  'Rivne_Oblast',
  'Sumy_Oblast',
  'Ternopil_Oblast',
  'Kharkiv_Oblast',
  'Kherson_Oblast',
  'Khmelnytskyi_Oblast',
  'Cherkasy_Oblast',
  'Chernivtsi_Oblast',
  'Chernihiv_Oblast',
  'Kyiv'],
 'predicted_gdp': [array([43416.574], dtype=float32),
  array([25596.617], dtype=float32),
  array([49831784.], dtype=float32),
  array([3.258329e+08], dtype=float32),
  array([24858146.], dtype=float32),
  array([249348.78], dtype=float32),
  array([-717731.8], dtype=float32),
  array([46669.582], dtype=float32),
  array([35730348.], dtype=float32),
  array([427689.3], dtype=float32),
  array([995587.94], dtype=float32),
  array([1451139.4], dtype=float32),
  array([

## Recurrent NN

In [38]:
percentage_error = results["percentage_error"]
np.mean(percentage_error) # 7.3% mean percentage error


7.291629

In [ ]:
# Preprocess the data

# load clean gdp data
gdp = pd.read_csv("data/clean_gdp.csv")

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(gdp), 765, 1076, 4))
y = np.zeros(len(gdp))

## Random Forest and other

In [4]:
# load the tabular data
data = pd.read_csv("data/tabular_data_ukraine.csv")

# turn the region column into a categorical variable using one hot encoding
data = pd.get_dummies(data, columns=["region"])

# training data contains years until 2021
train_data = data[data["year"] <= 2021]
test_data = data[data["year"] >= 2022]


In [6]:
# train a random forest model
from sklearn.ensemble import RandomForestRegressor

# drop year and gdp columns
X = train_data.drop(columns=["year", "real_gdp"])
y = train_data["real_gdp"]

# standardise gdp
# y_mean = y.mean()
# y_std = y.std()
# y = (y - y_mean) / y_std

y_max = y.max()
y = y / y_max

# take randomly 80% of the data for training
train_size = int(0.8 * len(train_data))
test_size = len(train_data) - train_size

train_indices = np.random.choice(len(train_data), train_size, replace=False)
test_indices = np.setdiff1d(np.arange(len(train_data)), train_indices)

X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]

# train the model
rf_model = RandomForestRegressor(n_estimators=1000, random_state = 0)
rf_model.fit(X_train, y_train)

# get the predictions
y_hat = rf_model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

0.018571927221726226
14.78645497172508


In [5]:
# fit a lasso regression model 
from sklearn.linear_model import Lasso

model = Lasso(alpha=0)

model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)


C:\Users\jakub\AppData\Local\Temp/ipykernel_316/1922239168.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train, y_train)
c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


0.01835888717836245
12.318338447454858


c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03104387917313304, tolerance: 0.0006106950436903142
  model = cd_fast.enet_coordinate_descent(


In [7]:
# fit a XGBoost model
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=0)

xgb_model.fit(X_train, y_train)

# get the predictions
y_hat = xgb_model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

0.01538115075906231
11.288293273346223


In [14]:
# fit a neural network model
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# turn X_train bool to integers
boolean_columns = [col for col in X_train.columns if X_train[col].dtype == bool]

for col in boolean_columns:
    X_train[col] = X_train[col].astype(int) 
 

# fit the model on the X_train and y_train
model = Sequential()
model.add(Dense(256, activation="relu", input_dim=X_train.shape[1]))  # More complex first layer
model.add(Dropout(0.3))  # Dropout to reduce overfitting
model.add(Dense(128, activation="relu"))  # Second layer
model.add(Dropout(0.2))  # Additional Dropout layer
model.add(Dense(64, activation="relu"))  # Third layer
model.add(Dense(32, activation="relu"))  # Fourth layer
model.add(Dropout(0.1))  # Dropout layer
model.add(Dense(16, activation="relu"))  # New additional layer
model.add(Dense(1)) 

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# get the predictions
boolean_columns = [col for col in X_test.columns if X_test[col].dtype == bool]
for col in boolean_columns:
    X_test[col] = X_test[col].astype(int)

y_hat = model.predict(X_test).flatten()

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/3771872762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/3771872762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/3771872762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

Epoch 1/100

5/5 [==============================] - 2s 73ms/step - loss: 61967528099840.0000 - mae: 5316903.0000 - val_loss: 70135830282240.0000 - val_mae: 5151166.0000
Epoch 2/100
5/5 [==============================] - 0s 14ms/step - loss: 61967184166912.0000 - mae: 5316874.5000 - val_loss: 70135326965760.0000 - val_mae: 5151121.5000
Epoch 3/100
5/5 [==============================] - 0s 13ms/step - loss: 61966458552320.0000 - mae: 5316811.0000 - val_loss: 70134311944192.0000 - val_mae: 5151032.5000
Epoch 4/100
5/5 [==============================] - 0s 12ms/step - loss: 61965162512384.0000 - mae: 5316689.0000 - val_loss: 70132260929536.0000 - val_mae: 5150853.0000
Epoch 5/100
5/5 [==============================] - 0s 12ms/step - loss: 61962226499584.0000 - mae: 5316441.5000 - val_loss: 70128213426176.0000 - val_mae: 5150499.0000
Epoch 6/100
5/5 [==============================] - 0s 12ms/step - loss: 61957075894272.0000 - mae: 5315957.0000 - val_loss: 70120185528320.0000 - val_mae: 5149

C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/3771872762.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/3771872762.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_15280/3771872762.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [7]:
# fit the model on the entire dataset, predict for year 2023
best_model = rf_model

# read the data
data = pd.read_csv("data/tabular_data_ukraine.csv")
data = pd.get_dummies(data, columns=["region"])

# get train and test
train_data = data[data["year"] <= 2021]
test_data = data[data["year"] == 2022]

X_train = train_data.drop(columns=["year", "gdp"])
y_train = train_data["gdp"]

X_test = test_data.drop(columns=["year", "gdp"])

# train 
best_model.fit(X_train, y_train)

# predict
y_hat = best_model.predict(X_test)

# add the predictions to the test_data
test_data = pd.read_csv("data/tabular_data_ukraine.csv")
test_data = test_data[test_data["year"] == 2022]

test_data["gdp_prediction"] = y_hat

results = test_data[["region", "gdp_prediction"]]


In [8]:
# compare results with gdp from the previous year
previous_year = pd.read_csv("data/tabular_data_ukraine.csv")
previous_year = previous_year[previous_year["year"] == 2021]
previous_year = previous_year[["region", "gdp"]]

# merge on region
results = results.merge(previous_year, on="region")

# compute change in percentages
results["change"] = (results["gdp_prediction"] - results["gdp"]) / results["gdp"]

results


,region,gdp_prediction,gdp,change
0,Vinnytsia_Oblast,2.593472e+06,4.337287e+06,-0.402052
1,Volyn_Oblast,2.142880e+06,1.915275e+06,0.118837
2,Dnipropetrovsk_Oblast,2.433512e+06,1.284456e+07,-0.810542
3,Donetsk_Oblast,3.229062e+06,6.400417e+06,-0.495492
4,Zhytomyr_Oblast,2.388363e+06,3.007165e+06,-0.205776
5,Zakarpattia_Oblast,2.183994e+06,2.096846e+06,0.041561
6,Zaporizhia_Oblast,2.167940e+06,5.481753e+06,-0.604517
7,Ivano-Frankivsk_Oblast,2.856070e+06,3.312617e+06,-0.137820
8,Kyiv_Oblast,4.064309e+06,7.395152e+06,-0.450409
9,Kirovohrad_Oblast,2.244913e+06,2.512884e+06,-0.106639


In [9]:
np.mean(results["change"]) # 23% drop

(results["gdp_prediction"].sum() - results["gdp"].sum()) / results["gdp"].sum() # 31.8% drop, World Bank esimates 29.1%

-0.3221699063478681

In [30]:
# select region_Dnipropetrovsk_Oblast == 1
subset = data[data["region_Dnipropetrovsk_Oblast"] == 1]
subset

,year,gdp,nearnad_snow_cov_mean,nearnad_snow_cov_median,nearnad_snow_cov_std,nearnad_snow_free_mean,nearnad_snow_free_median,nearnad_snow_free_std,offnad_snow_cov_mean,offnad_snow_cov_median,...,region_Odessa_Oblast,region_Poltava_Oblast,region_Rivne_Oblast,region_Sumy_Oblast,region_Ternopil_Oblast,region_Vinnytsia_Oblast,region_Volyn_Oblast,region_Zakarpattia_Oblast,region_Zaporizhia_Oblast,region_Zhytomyr_Oblast
2,2012,1.479700e+07,2.647959,0.0,46.242333,1.184331,0.0,43.000168,2.546648,0.0,...,False,False,False,False,False,False,False,False,False,False
27,2013,1.469342e+07,2.343649,0.0,33.583948,0.961109,0.0,30.325981,1.299532,0.0,...,False,False,False,False,False,False,False,False,False,False
52,2014,1.397344e+07,3.100126,0.0,40.716915,0.974721,0.0,25.166740,2.828148,0.0,...,False,False,False,False,False,False,False,False,False,False
77,2015,1.261802e+07,2.467506,0.0,50.238729,0.930540,0.0,26.208755,2.504813,0.0,...,False,False,False,False,False,False,False,False,False,False
102,2016,1.241613e+07,3.240427,0.0,31.904961,1.048079,0.0,41.651880,1.986214,0.0,...,False,False,False,False,False,False,False,False,False,False
127,2017,1.266445e+07,2.544386,0.0,39.548876,1.243090,0.0,52.883580,2.503581,0.0,...,False,False,False,False,False,False,False,False,False,False
152,2018,1.298107e+07,3.188704,0.0,55.825475,1.250778,0.0,50.561350,2.398645,0.0,...,False,False,False,False,False,False,False,False,False,False
177,2019,1.346136e+07,3.057841,0.0,51.197801,1.254185,0.0,41.082681,2.261230,0.0,...,False,False,False,False,False,False,False,False,False,False
202,2020,1.251907e+07,3.204058,0.0,67.320126,1.225739,0.0,30.918232,1.514519,0.0,...,False,False,False,False,False,False,False,False,False,False
227,2021,1.284456e+07,4.045430,0.0,45.211584,1.338012,0.0,31.280639,3.359127,0.0,...,False,False,False,False,False,False,False,False,False,False


In [46]:
train_data

,year,gdp,nearnad_snow_cov_mean,nearnad_snow_cov_median,nearnad_snow_cov_std,nearnad_snow_free_mean,nearnad_snow_free_median,nearnad_snow_free_std,offnad_snow_cov_mean,offnad_snow_cov_median,...,region_Odessa_Oblast,region_Poltava_Oblast,region_Rivne_Oblast,region_Sumy_Oblast,region_Ternopil_Oblast,region_Vinnytsia_Oblast,region_Volyn_Oblast,region_Zakarpattia_Oblast,region_Zaporizhia_Oblast,region_Zhytomyr_Oblast
0,2012,100.000000,0.945420,0.0,13.503158,0.301091,0.0,3.865808,1.307311,0.0,...,False,False,False,False,False,True,False,False,False,False
1,2012,100.000000,0.767207,0.0,6.420684,0.102880,0.0,2.273735,0.682606,0.0,...,False,False,False,False,False,False,True,False,False,False
2,2012,100.000000,2.647959,0.0,46.242333,1.184331,0.0,43.000168,2.546648,0.0,...,False,False,False,False,False,False,False,False,False,False
3,2012,100.000000,3.566429,0.0,39.045082,1.317057,0.0,14.056379,3.413734,0.0,...,False,False,False,False,False,False,False,False,False,False
4,2012,100.000000,0.717199,0.0,11.332430,0.201979,0.0,3.190241,1.358591,0.0,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2021,110.225016,0.622373,0.0,8.658013,0.196125,0.0,3.930659,0.655630,0.0,...,False,False,False,False,False,False,False,False,False,False
246,2021,105.572198,1.308531,0.0,73.238358,0.349940,0.0,38.114947,1.509102,0.0,...,False,False,False,False,False,False,False,False,False,False
247,2021,104.896846,0.363287,0.0,4.754161,0.155258,0.0,3.675048,0.342800,0.0,...,False,False,False,False,False,False,False,False,False,False
248,2021,97.226457,0.404712,0.0,7.745696,0.107963,0.0,3.970843,0.414831,0.0,...,False,False,False,False,False,False,False,False,False,False
